<a href="https://colab.research.google.com/github/esdrasantos/etl-faturas-nubank/blob/main/Script_Proc_Faturas_Nu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# installs

In [ ]:
!pip install pandas
!pip install pdfplumber

# classifier

In [ ]:
import re

purchase_category_dict = {
      'Shopping e ecommerce': {
          'Mercadolivre', 'Livraria', 'Mp', 'Mlp', 'Americanas', 'Pdv', 'So Ler',
          'Pag', 'Hna', 'Amazon', 'Renner', 'Zara', 'Acessorios', 'Melissa',
          'Shopee', 'Lolja', 'Saoleopoldomeias', 'Marisa', 'Prata', 'Moda',
          'Bazar', 'Brecho', 'Shopping', 'Magalu', 'Shein', 'Torra Torra', 'Marbela', 'Festa', 'Presente', 'Armarinhos'
      },
      'Ifood': {
          'ifd', 'ifood'
      },
      'Restaurantes': {
          'The Waffle King', 'Coffee', 'Divino Fogao', 'Pub', 'Restaurante',
          'Ceu da Boca', 'Bar', 'Cafe', 'Bistr', 'Ru Unisinos', 'Bocattino',
          'Mokai', 'Mercearia', 'Pampa', 'Sorveteria', 'Mood', 'Cantina',
          'Ice Cream'
      },
      'Lanches e Conveniencias': {
          'Station', 'Posto', 'Lanche', 'Happy Station', 'Florinda',
          'Conveniencias'
      },
      'Mercados': {
          'Macromix', 'Padaria', 'Supermercado', 'Bourbon', 'Atacado',
          'Mercado', 'Havan', 'Zaffari', 'Unidasul', 'Frutas', 'Minimercado',
          'Casa dos Cereais', 'Armazem 88'
      },
      'Shows e cinema': {
          'Tickets', 'Cinema', 'Opiniao', 'Sympla'
      },
      'Transporte': {
          'Palmares', 'Uber*', 'Uber', 'Citral', 'Via Sul'
      },
      'Saude e Farmacia': {
          'Pharma', 'Odonto', 'Farmacia', 'Atendvip', 'Oi Digital',
          'Panvel', 'Farma', 'Gabriel', 'Betterme'
      },
      'Beleza': {
          'Opus', 'Miriammf', 'Boticario', 'Beleza'
      },
      'Lavanderia': {
          'Cicclo', 'Begin'
      },
      'Assinaturas e Servicos': {
          'Prime', 'Tembici', 'Netflix', 'Spotify', 'Vogue'
      },
      'Casa': {
          'Imobiliaria', 'Net', 'Rge', 'Claro', 'Flex'
      },
      'Faculdade': {
          'Matricula', 'Unisinos'
      },
      'Apostas': {
          'Sorte'
      },
      'Bar': {
          'Brecho do Futebol', 'Majestic', 'Quiosque', 'Drink'
      },
      'Tecnologia e Projetos': {
          'Openai'
      },
      'Viagem': {
          'Airbnb'
      }
}

def classify_purchases(purchase_description):
    for category, keywords in purchase_category_dict.items():
        for keyword in keywords:
            if keyword.lower() in purchase_description.lower():
                return category
    return 'Outro'


# cleaning

In [ ]:
import pandas as pd
import pdfplumber as pdfplumber
import re

# Função para extrair colunas com base no padrão Data, Compra e Valor
def extract_columns_from_row(row):
    # Padrão para Data: Dois dígitos seguidos por três letras (ex: '04 JUL')
    date_pattern = r'^\d{2} \w{3}'
    # Padrão para Valor: Um valor monetário no formato R$ xxx,xx
    value_pattern = r'R\$ \d{1,3}(?:\.\d{3})*,\d{2}'

    # Tenta encontrar Data no início da linha
    date_match = re.match(date_pattern, row)
    if date_match:
        date = date_match.group(0)  # '04 JUL'
        # Tenta encontrar o Valor no final da linha
        value_match = re.search(value_pattern, row)
        if value_match:
            value = value_match.group(0)  # 'R$ 52,51'
            # O que sobrou entre a Data e o Valor é a descrição da compra
            if date is not None and value_match is not None:
              purchase = row[len(date):value_match.start()].strip()
              return [date, purchase, value]
    return None


def read_pdf_as_dataframe(pdf_path):
    # Lista para armazenar as tabelas de todas as páginas
    all_tables = []

    # Abre o arquivo PDF
    with pdfplumber.open(pdf_path) as pdf:
        # Itera sobre cada página do PDF
        for page_num, page in enumerate(pdf.pages):
            # Extrai a tabela da página
            table = page.extract_table()

            # Pula a primeira página
            if page_num == 1:
                continue

            # Se uma tabela for encontrada na página, adicione à lista
            if table:
                # Converte a tabela para DataFrame e adiciona à lista
                proc_data = [extract_columns_from_row(row[0]) for row in table if row]
                proc_data = list(filter(None, proc_data))
                df = pd.DataFrame(proc_data)
                all_tables.append(df)
                print(f"Tabela extraída da página {page_num + 1}")

    return pd.concat(all_tables, ignore_index=True)


def remove_nan_columns(dataframe):
    for column in dataframe.columns:
        if dataframe[column].isnull().all():
            dataframe.drop(columns=column, axis=1, inplace=True)


def remove_nan_lines(dataframe):
    dataframe.dropna(inplace=True)


def remove_lines(dataframe, condition_mask):
    return (dataframe[condition_mask]
            .reset_index(drop=True))


def reset_dataframe_column_indexes(dataframe):
    length = len(dataframe.columns)
    dataframe.columns = range(length)
    dataframe.rename(columns={0: DATE,
                              1: PURCHASE,
                              2: VALUE},
                     inplace=True)


# constants

In [ ]:
DATE = 'Data'
PURCHASE = 'Compra'
VALUE = 'Valor'
PAYMENT = 'Pagamento'
REFUND = 'Estorno'
CATEGORY = 'Categoria'

# mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# main

In [ ]:
import warnings
import pandas as pd
import os

def main_function(event, context):

    warnings.simplefilter(action='ignore')
    pdf_folder = '/content/drive/MyDrive/faturas_nu/'
    output_dir = '/content/drive/MyDrive/faturas_nu_csv/'
    pdf_files  = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

    for pdf in pdf_files:
        print(f'Processing {pdf}')
        pdf_path = f'{pdf_folder}/{pdf}'
        dataframe = read_pdf_as_dataframe(pdf_path)
        dataframe.columns = [DATE, PURCHASE, VALUE]

        # Máscara que identifica linhas de 'Pagamento' na tabela de compras
        is_payment_line = (dataframe[PURCHASE]
                        .str.
                        startswith(PAYMENT))

        # Prepara uma tabela apenas com as linhas de 'Pagamento'
        payments = remove_lines(dataframe, is_payment_line)

        # Remove linhas de pagamento de entre as linhas de compras
        dataframe = remove_lines(dataframe, ~is_payment_line)
        dataframe[CATEGORY] = dataframe[PURCHASE].apply(classify_purchases) # É facil 'estender' os objetos em python

        print(dataframe.head())

        # Define o caminho de saída para o CSV
        csv_file_name = pdf.replace('.pdf', '.csv')  # Troca a extensão para .csv

        os.makedirs(os.path.dirname(output_dir), exist_ok=True)

        output_path = os.path.join(output_dir, csv_file_name)

        # Salva o DataFrame como CSV
        dataframe.to_csv(output_path, index=False)

main_function("","")

Processing Nubank_2025-04-11.pdf
Tabela extraída da página 5
Tabela extraída da página 6
Tabela extraída da página 7
Tabela extraída da página 8
     Data                                Compra     Valor  \
0  04 MAR          Panvelfilial11 - Parcela 2/3  R$ 79,98   
1  04 MAR   Mercadolivre*3produto - Parcela 3/3  R$ 16,54   
2  04 MAR  Nh Comercio de Cosmeti - Parcela 2/4  R$ 35,21   
3  04 MAR      Amazon Marketplace - Parcela 5/7  R$ 53,28   
4  04 MAR                                Gelmat  R$ 15,00   

              Categoria  
0      Saude e Farmacia  
1  Shopping e ecommerce  
2                 Outro  
3  Shopping e ecommerce  
4                 Outro  
Processing Nubank_2025-03-11.pdf
Tabela extraída da página 5
Tabela extraída da página 6
Tabela extraída da página 7
Tabela extraída da página 8
Tabela extraída da página 9
Tabela extraída da página 10
     Data                               Compra     Valor             Categoria
0  04 FEV       Saoleopoldomeias - Parcela 4/4  R$ 